# API RAPID AMAZON

### Endpoints Best Sellers AMAZON

In [2]:
import os
import requests
import pandas as pd
from dotenv import load_dotenv
from datetime import datetime
import logging

# Configurar o logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler("amazon_data.log"),
        logging.StreamHandler()
    ]
)

# Carregar variáveis de ambiente do arquivo .env
load_dotenv()

# Obter a chave da API a partir das variáveis de ambiente
rapidapi_key = os.getenv("RAPIDAPI_KEY")

# Dicionário de categorias
categories_dict = {
    "Alle afdelingen": "aps",
    "Amazon Warehouse": "warehouse-deals",
    "Amazon-apparaten": "amazon-devices",
    "Auto en motor": "automotive",
    "Babyproducten": "baby",
    "Beauty en persoonlijke verzorging": "beauty",
    "Black Friday": "black-friday",
    "Boeken": "stripbooks",
    "Cadeaubonnen": "gift-cards",
    "Cd's en lp's": "popular",
    "Elektronica": "electronics",
    "Films en tv": "dvd",
    "Gezondheid en persoonlijke verzorging": "hpc",
    "Huisdierbenodigdheden": "pets",
    "Kantoorproducten": "office-products",
    "Kindle Store": "digital-text",
    "Kleding, schoenen en sieraden": "fashion",
    "Klussen en gereedschap": "home-improvement",
    "Levensmiddelen": "grocery",
    "Muziekinstrumenten": "mi",
    "Overig": "misc",
    "Prime Video": "instant-video",
    "Software": "software",
    "Speelgoed en spellen": "toys",
    "Sport en outdoor": "sports",
    "Tuin, terras en gazon": "outdoor",
    "Videogames": "videogames",
    "Wonen en keuken": "home",
    "Zakelijk, industrie en wetenschap": "industrial"
}

# Lista de categorias (category_id)
categories = list(categories_dict.values())

# Definir a URL da API
url = "https://real-time-amazon-data.p.rapidapi.com/best-sellers"

# Definir os cabeçalhos da solicitação
headers = {
    "x-rapidapi-key": rapidapi_key,
    "x-rapidapi-host": "real-time-amazon-data.p.rapidapi.com"
}

# Inicializar lista para armazenar DataFrames
all_dataframes = []

# Função para processar cada categoria
def process_category(category_id, category_name):
    querystring = {"category": category_id, "type": "BEST_SELLERS", "page": "1", "country": "NL"}
    
    try:
        # Fazer a solicitação à API
        response = requests.get(url, headers=headers, params=querystring)
        response.raise_for_status()  # Levantar exceção para códigos de erro HTTP

        data = response.json()

        # Extrair a data atual
        today = datetime.today().strftime('%Y-%m-%d')

        # Normalizar os dados JSON e converter para DataFrame
        best_sellers = data['data']['best_sellers']
        df = pd.json_normalize(best_sellers)

        # Verificar se a coluna 'product_price' existe e processar
        if 'product_price' in df.columns:
            # Remover o símbolo da moeda euro do campo "product_price" e converter para float
            df['product_price'] = df['product_price'].str.replace('€', '').str.replace(',', '.').str.strip().astype(float)
        else:
            # Adicionar a coluna 'product_price' com valores NaN se não existir
            df['product_price'] = float('nan')

        # Adicionar a coluna "name_category", "category_id" e "date" como as últimas colunas
        df.insert(len(df.columns), 'name_category', category_name)
        df.insert(len(df.columns), 'category_id', category_id)
        df.insert(len(df.columns), 'date', today)

        # Adicionar DataFrame à lista
        all_dataframes.append(df)

        # Definir o nome do arquivo XLSX individual
        file_name = f"{category_name}_Amazon_bestseller_{today}.xlsx"

        # Definir o diretório de salvamento com as duas pastas adicionais
        save_directory = os.path.join(os.getcwd(), 'output', 'rapid_amazon_best_sellers')

        # Certifique-se de que o diretório existe
        os.makedirs(save_directory, exist_ok=True)

        # Definir o caminho completo do arquivo
        file_path = os.path.join(save_directory, file_name)

        # Salvar o DataFrame em um arquivo Excel individual
        df.to_excel(file_path, index=False)

        logging.info(f"Arquivo salvo como: {file_name}")

    except requests.exceptions.HTTPError as http_err:
        logging.error(f"HTTP error ao processar a categoria {category_name}: {http_err}")
    except Exception as err:
        logging.error(f"Erro ao processar a categoria {category_name}: {err}")

# Processar cada categoria na lista
for name_category, category_id in categories_dict.items():
    process_category(category_id, name_category)

# Concatenar todos os DataFrames em um único DataFrame
if all_dataframes:
    final_df = pd.concat(all_dataframes, ignore_index=True)

    # Definir o nome do arquivo XLSX final
    final_file_name = f"All_Categories_Amazon_bestseller_{datetime.today().strftime('%Y-%m-%d')}.xlsx"
    final_file_path = os.path.join(os.getcwd(), 'output', 'rapid_amazon_best_sellers', final_file_name)

    # Salvar o DataFrame final em um arquivo Excel
    final_df.to_excel(final_file_path, index=False)

    logging.info(f"Arquivo XLSX final salvo como: {final_file_path}")
else:
    logging.warning("Nenhum dado foi processado para salvar no arquivo final.")


2024-08-09 10:58:12,030 - ERROR - HTTP error ao processar a categoria Alle afdelingen: 429 Client Error: Too Many Requests for url: https://real-time-amazon-data.p.rapidapi.com/best-sellers?category=aps&type=BEST_SELLERS&page=1&country=NL
2024-08-09 10:58:12,072 - ERROR - HTTP error ao processar a categoria Amazon Warehouse: 429 Client Error: Too Many Requests for url: https://real-time-amazon-data.p.rapidapi.com/best-sellers?category=warehouse-deals&type=BEST_SELLERS&page=1&country=NL
2024-08-09 10:58:12,124 - ERROR - HTTP error ao processar a categoria Amazon-apparaten: 429 Client Error: Too Many Requests for url: https://real-time-amazon-data.p.rapidapi.com/best-sellers?category=amazon-devices&type=BEST_SELLERS&page=1&country=NL
2024-08-09 10:58:12,175 - ERROR - HTTP error ao processar a categoria Auto en motor: 429 Client Error: Too Many Requests for url: https://real-time-amazon-data.p.rapidapi.com/best-sellers?category=automotive&type=BEST_SELLERS&page=1&country=NL
2024-08-09 10: